# 실습 09: NER 기반 핵심 정보 추출

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leecks1119/document_ai_lecture/blob/master/notebooks/Lab09_NER정보추출.ipynb)

## 🎯 학습 목표
- NER(Named Entity Recognition) 이해
- 규칙 기반 정보 추출
- 실전 문서에서 핵심 정보 자동 추출

## ⏱️ 소요 시간: 30분
## 📊 난이도: ⭐⭐⭐☆☆


In [ ]:
# 한글 폰트 설치 및 설정 (Colab용)
!apt-get install -y fonts-nanum fonts-nanum-coding fonts-nanum-extra
!fc-cache -fv

# matplotlib 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rc('font', family=fontprop.get_name())
plt.rc('axes', unicode_minus=False)

print("✅ 한글 폰트 설정 완료!")


In [ ]:
# 환경 설정
!pip install -q git+https://github.com/leecks1119/document_ai_lecture.git paddlepaddle paddleocr
!apt-get install -y tesseract-ocr tesseract-ocr-kor


In [ ]:
# 계약서 샘플 이미지 생성
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# 한글 폰트 로드
try:
    font_title = ImageFont.truetype("C:\\Windows\\Fonts\\malgunbd.ttf", 32)
    font_label = ImageFont.truetype("C:\\Windows\\Fonts\\malgun.ttf", 16)
    font_data = ImageFont.truetype("C:\\Windows\\Fonts\\malgun.ttf", 18)
except:
    try:
        font_title = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf", 32)
        font_label = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothic.ttf", 16)
        font_data = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothic.ttf", 18)
    except:
        font_title = ImageFont.load_default()
        font_label = ImageFont.load_default()
        font_data = ImageFont.load_default()

# 계약서 이미지 생성
img = Image.new('RGB', (800, 600), color='white')
draw = ImageDraw.Draw(img)

# 제목
draw.text((300, 30), "용역 계약서", fill='black', font=font_title)

# 계약 정보
y_pos = 100
contract_data = [
    ("계약일자:", "2025년 3월 1일"),
    ("발주자:", "삼성전자 주식회사"),
    ("담당자:", "김철수 부장"),
    ("연락처:", "02-2000-1234"),
    ("이메일:", "chulsoo.kim@samsung.com"),
    ("", ""),
    ("수주자:", "(주)테크솔루션"),
    ("연락처:", "031-456-7890"),
    ("이메일:", "contact@techsolution.com"),
    ("", ""),
    ("계약금액:", "50,000,000원"),
    ("입금계좌:", "123-456-789012"),
]

for label, value in contract_data:
    if label:
        draw.text((50, y_pos), label, fill='gray', font=font_label)
        draw.text((200, y_pos), value, fill='black', font=font_data)
    y_pos += 35

img.save('contract_doc.jpg')
print("✅ 계약서 이미지 생성 완료!")
img


In [ ]:
# PaddleOCR로 텍스트와 위치 정보 추출
from paddleocr import PaddleOCR
import cv2

print("📄 OCR 실행 중...")
ocr = PaddleOCR(use_textline_orientation=True, lang='korean')
result = ocr.predict('contract_doc.jpg')

# OCR 결과 추출
ocr_result = result[0]
texts = ocr_result['rec_texts']
scores = ocr_result['rec_scores']
boxes = ocr_result['dt_polys']

print(f"✅ OCR 완료: {len(texts)}개의 텍스트 영역 검출\n")
print("="*70)
print("검출된 텍스트:")
print("="*70)
for i, (text, score) in enumerate(zip(texts, scores), 1):
    print(f"{i:2d}. {text:30s} (신뢰도: {score:.2%})")


In [ ]:
# 이미지에 NER 결과 시각화 (엔티티 타입별 색상 표시)
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import cv2
import numpy as np

# 엔티티 타입별 색상 정의
entity_colors = {
    'DATE': (255, 100, 100),       # 빨강
    'ORG': (100, 255, 100),        # 초록
    'PERSON': (100, 100, 255),     # 파랑
    'PHONE': (255, 255, 100),      # 노랑
    'EMAIL': (255, 100, 255),      # 마젠타
    'MONEY': (100, 255, 255),      # 시안
    'ACCOUNT': (200, 150, 100),    # 갈색
    'OTHER': (150, 150, 150),      # 회색
}

# 한글 폰트 설정 (코랩 환경용)
import os

def get_korean_font():
    """코랩 환경에서 한글 폰트를 찾아서 반환"""
    # 코랩에서 설치된 나눔폰트 경로들
    font_paths = [
        '/usr/share/fonts/truetype/nanum/NanumGothic.ttf',
        '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf',
        '/usr/share/fonts/truetype/nanum/NanumMyeongjo.ttf',
        '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc',
        '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf'  # fallback
    ]
    
    for font_path in font_paths:
        if os.path.exists(font_path):
            try:
                return fm.FontProperties(fname=font_path)
            except:
                continue
    
    # 폰트를 찾지 못한 경우 기본 폰트 사용
    print("⚠️  한글 폰트를 찾을 수 없어 기본 폰트를 사용합니다.")
    return fm.FontProperties()

# 폰트 설정
fontprop = get_korean_font()

# matplotlib 전역 설정
plt.rc('font', family=fontprop.get_name())
plt.rc('axes', unicode_minus=False)

# 더미 이미지에 맞는 바운딩 박스 좌표 설정
# 더미 텍스트 위치:
# - "Name: Kim Sang Hee" at (50, 100)
# - "Company: Samsung" at (50, 150) 
# - "Date: 2024-01-15" at (50, 200)
# - "Phone: 010-1234-5678" at (50, 250)

ocr_with_ner = [
    {
        'bbox': [[200, 85], [350, 85], [350, 115], [200, 115]],  # "Kim Sang Hee" 영역
        'entity_type': 'PERSON',
        'text': 'Kim Sang Hee'
    },
    {
        'bbox': [[200, 135], [300, 135], [300, 165], [200, 165]],  # "Samsung" 영역
        'entity_type': 'ORG',
        'text': 'Samsung'
    },
    {
        'bbox': [[200, 185], [350, 185], [350, 215], [200, 215]],  # "2024-01-15" 영역
        'entity_type': 'DATE',
        'text': '2024-01-15'
    },
    {
        'bbox': [[200, 235], [400, 235], [400, 265], [200, 265]],  # "010-1234-5678" 영역
        'entity_type': 'PHONE',
        'text': '010-1234-5678'
    }
]

# 이미지 로드 (파일이 없으면 더미 이미지 생성)
try:
    img = cv2.imread('contract_doc.jpg')
    if img is None:
        raise FileNotFoundError("이미지 파일을 찾을 수 없습니다")
except:
    # 더미 이미지 생성 (바운딩 박스에 맞는 크기로 조정)
    img = np.ones((400, 600, 3), dtype=np.uint8) * 255
    print("⚠️  contract_doc.jpg 파일이 없어서 더미 이미지를 생성했습니다.")
    
    # 더미 텍스트 추가 (시각적 확인용)
    cv2.putText(img, "Sample Document", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.putText(img, "Name: Kim Sang Hee", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
    cv2.putText(img, "Company: Samsung", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
    cv2.putText(img, "Date: 2024-01-15", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
    cv2.putText(img, "Phone: 010-1234-5678", (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
    
    # 바운딩 박스 영역을 시각적으로 표시 (디버깅용)
    cv2.rectangle(img, (200, 85), (350, 115), (255, 0, 0), 1)   # PERSON 영역
    cv2.rectangle(img, (200, 135), (300, 165), (0, 255, 0), 1)  # ORG 영역  
    cv2.rectangle(img, (200, 185), (350, 215), (0, 0, 255), 1)  # DATE 영역
    cv2.rectangle(img, (200, 235), (400, 265), (255, 255, 0), 1) # PHONE 영역

img_annotated = img.copy()

# 한글 폰트 로드 (레이블용 - PIL용)
try:
    from PIL import Image, ImageDraw, ImageFont
    
    # 코랩에서 사용 가능한 폰트 경로들
    pil_font_paths = [
        '/usr/share/fonts/truetype/nanum/NanumGothic.ttf',
        '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf',
        '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
    ]
    
    label_font = None
    for font_path in pil_font_paths:
        if os.path.exists(font_path):
            try:
                label_font = ImageFont.truetype(font_path, 14)
                break
            except:
                continue
    
    if label_font is None:
        label_font = ImageFont.load_default()
        print("⚠️  PIL용 한글 폰트를 찾을 수 없어 기본 폰트를 사용합니다.")
        
except ImportError:
    print("⚠️  PIL이 설치되지 않았습니다. 텍스트 레이블이 제대로 표시되지 않을 수 있습니다.")
    label_font = None

# OpenCV로 직접 처리 (PIL 변환 제거)

# 각 텍스트 영역에 바운딩 박스와 라벨 표시
for item in ocr_with_ner:
    bbox = item['bbox']
    entity_type = item['entity_type']
    text = item['text']
    
    # 색상 선택
    color = entity_colors.get(entity_type, (150, 150, 150))
    
    # 바운딩 박스 좌표 (OpenCV 형식: [x, y])
    points = np.array(bbox, dtype=np.int32)
    x_min, y_min = points[:, 0].min(), points[:, 1].min()
    x_max, y_max = points[:, 0].max(), points[:, 1].max()
    
    print(f"🔍 {entity_type}: {text} -> bbox: ({x_min}, {y_min}) to ({x_max}, {y_max})")
    
    # OpenCV로 박스 그리기 (BGR 색상 순서) - 더 굵게
    cv2.polylines(img_annotated, [points], True, color, 4)
    
    # 라벨 배경 박스 그리기 (OpenCV로)
    label = f"{entity_type}"
    label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
    
    # 라벨 배경 사각형
    cv2.rectangle(img_annotated, 
                  (x_min, y_min - label_size[1] - 10), 
                  (x_min + label_size[0] + 4, y_min - 2), 
                  color, -1)
    
    # 라벨 텍스트
    cv2.putText(img_annotated, label, 
                (x_min + 2, y_min - 5), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

# PIL 변환 제거 (OpenCV로 직접 처리)

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# 원본 이미지
axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0].set_title('원본 이미지', fontsize=14, fontproperties=fontprop, pad=15)
axes[0].axis('off')

# NER 결과 표시된 이미지
axes[1].imshow(cv2.cvtColor(img_annotated, cv2.COLOR_BGR2RGB))
axes[1].set_title('NER 결과 매핑 (엔티티 타입별 색상)', fontsize=14, fontproperties=fontprop, pad=15)
axes[1].axis('off')

plt.tight_layout()
plt.show()

# 범례 표시
print("\n" + "="*70)
print("🎨 엔티티 타입별 색상")
print("="*70)
for entity_type, color in entity_colors.items():
    if any(item['entity_type'] == entity_type for item in ocr_with_ner):
        rgb = f"RGB{color}"
        print(f"  {entity_type:12s} : {rgb}")

print("\n✅ NER 매핑 시각화 완료!")


In [ ]:
# 구조화된 데이터로 변환 및 저장
import pandas as pd

# test.py에서 정의된 ocr_with_ner 데이터가 필요합니다
# 만약 실행되지 않았다면 샘플 데이터를 사용합니다
if 'ocr_with_ner' not in globals():
    print("⚠️  ocr_with_ner 데이터가 없습니다. test.py를 먼저 실행해주세요.")
    print("샘플 데이터를 사용합니다...")
    
    ocr_with_ner = [
        {
            'bbox': [[200, 85], [350, 85], [350, 115], [200, 115]],
            'entity_type': 'PERSON',
            'text': 'Kim Sang Hee',
            'confidence': 0.95
        },
        {
            'bbox': [[200, 135], [300, 135], [300, 165], [200, 165]],
            'entity_type': 'ORG',
            'text': 'Samsung',
            'confidence': 0.92
        },
        {
            'bbox': [[200, 185], [350, 185], [350, 215], [200, 215]],
            'entity_type': 'DATE',
            'text': '2024-01-15',
            'confidence': 0.98
        },
        {
            'bbox': [[200, 235], [400, 235], [400, 265], [200, 265]],
            'entity_type': 'PHONE',
            'text': '010-1234-5678',
            'confidence': 0.94
        }
    ]

# 엔티티 타입별로 그룹화
data_by_type = {}
for item in ocr_with_ner:
    entity_type = item['entity_type']
    if entity_type != 'OTHER':
        if entity_type not in data_by_type:
            data_by_type[entity_type] = []
        data_by_type[entity_type].append(item['text'])

print("\n" + "="*70)
print("📋 추출된 정보 요약")
print("="*70)
for entity_type, values in data_by_type.items():
    print(f"\n[{entity_type}]")
    for value in values:
        print(f"  • {value}")

# DataFrame으로 저장
data = []
for item in ocr_with_ner:
    if item['entity_type'] != 'OTHER':
        # test.py의 데이터 구조에 맞게 수정
        data.append({
            '엔티티 타입': item['entity_type'],
            '텍스트': item['text'],
            'OCR 신뢰도': f"{item.get('confidence', 0.95):.2%}",  # confidence 필드 사용
            '바운딩 박스': f"({item['bbox'][0][0]}, {item['bbox'][0][1]}) - ({item['bbox'][2][0]}, {item['bbox'][2][1]})"
        })

df = pd.DataFrame(data)
print("\n" + "="*70)
print("📊 전체 추출 결과")
print("="*70)
print(df.to_string(index=False))

# CSV로 저장
df.to_csv('contract_entities.csv', index=False, encoding='utf-8-sig')
print("\n✅ CSV 저장 완료: contract_entities.csv")


In [ ]:
# NER 시스템으로 각 텍스트 영역 분류
from docai_course.ner import UnifiedNERSystem

ner = UnifiedNERSystem()

# 각 텍스트에 NER 적용
ocr_with_ner = []
for text, score, bbox in zip(texts, scores, boxes):
    # NER 분석
    entities = ner.rule_based_ner(text)
    
    # 가장 신뢰도 높은 엔티티 선택
    if entities:
        entity_type = entities[0]['entity']
        entity_conf = entities[0]['confidence']
    else:
        entity_type = 'OTHER'
        entity_conf = 0.0
    
    ocr_with_ner.append({
        'text': text,
        'bbox': bbox,
        'ocr_score': score,
        'entity_type': entity_type,
        'entity_conf': entity_conf
    })

print("\n" + "="*70)
print("📊 NER 분류 결과")
print("="*70)
for i, item in enumerate(ocr_with_ner, 1):
    print(f"{i:2d}. [{item['entity_type']:15s}] {item['text']:30s} (NER신뢰도: {item['entity_conf']:.2%})")
